In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import time
from tqdm.auto import tqdm
import random
import copy
from joblib import Parallel, delayed
import os
import shutil
import subprocess
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align.Applications import MuscleCommandline
import re
import shutil
from Comparative_Analysis import Sequence_Analysis_Routines as sar
from Comparative_Analysis import Utilities as util
from Comparative_Analysis import Alignment as align
from Comparative_Analysis import Blast_Functions as blastfn
from scipy.stats import chi2
from IPython import display
num_cores = 16
core_numbers = list(range(1, num_cores+1))

project_dir = 'D:/Project_Data/Project_8'
output_dir = project_dir + '/Test'
datasets_dir = project_dir + '/Datasets'
seq_dir = datasets_dir + '/Actinobacteria_Ref_Rep_Lev_Complete'
seed_species = 'NZ_CP102342.1'
tb_species = 'NC_000962.3' 
seed_dir = project_dir + '/R_Scape_Results_Smegmatis'

search_region = 'U'

In [2]:
with open(output_dir + '/gene_info_dict.pkl', 'rb') as f:
    gene_info_dict = pickle.load(f) 
with open(output_dir + '/names_dict.pkl', 'rb') as f:
    names_dict = pickle.load(f) 
with open(output_dir + '/filename_dict.pkl', 'rb') as f:
    filename_dict = pickle.load(f) 

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Project_Data/Project_8/Test/gene_info_dict.pkl'

In [3]:
def display_structure(locus):
    if os.path.exists(screen_dir + '/' + locus + '/rscape_3.cacofold.R2R.sto.svg'):
                return display.SVG(filename =screen_dir + '/' + locus + '/rscape_3.cacofold.R2R.sto.svg')

In [4]:
def find_tb_region_info(start, stop, seq_from, seq_to, search_region):
    current_overlap = 0
    tb_genome_record = next(SeqIO.parse(seq_dir + '/' + tb_filename, "genbank"))
    for feature in tb_genome_record.features:
        if not(feature.type == 'gene'):
            a = feature.qualifiers
            if a.get("locus_tag")!= None:
                (start_f, stop_f, strand_f) = (int(feature.location.start),int(feature.location.end),int(feature.location.strand))
                if stop_f - start_f  > 100000:   #Something wrong!
                    continue
                start_1 = min(stop, start)
                stop_1 = max(stop, start)
                if stop_1 > start_f and start_1 < stop_f:
                    overlap = (min(stop_f, stop_1) - max(start_f, start_1)) / (stop_1 - start_1)
                    if overlap > current_overlap:
                        current_overlap = overlap
                        feature_strand = strand_f
    if search_region == 'U':
        if feature_strand == 1:
            output_info = (start - 500 + seq_from, start - 500 + seq_to)
        else:
            output_info = (stop + 500 - seq_to, stop + 500 - seq_from)
    else:
        if feature_strand == 1:
            output_info = (stop + 500 - seq_to, stop + 500 - seq_from)
        else:
            output_info = (start - 500 + seq_from, start - 500 + seq_to)
    return output_info

In [27]:
seed_dir = project_dir + '/R_Scape_Results_MTb'
#seed_dir = project_dir + '/R_Scape_Results_Smegmatis'
#seed_species = 'NZ_CP102342.1'
tb_species = 'NC_000962.3' 
#tb_species = seed_species
seed_species = tb_species
seed_filename = filename_dict[seed_species]
tb_filename = filename_dict[tb_species]
seed_product_dict = {}
seed_position_dict = {}
tb_position_dict = {}
seed_genome_record = next(SeqIO.parse(seq_dir + '/' + seed_filename, "genbank"))
tb_genome_record = next(SeqIO.parse(seq_dir + '/' + tb_filename, "genbank"))
for feature in seed_genome_record.features:
    if not(feature.type == 'gene'):
        a = feature.qualifiers
        if a.get("locus_tag")!= None:
            seed_position_dict[a.get("locus_tag")[0]] = (int(feature.location.start),int(feature.location.end),int(feature.location.strand))
            if a.get("locus_tag")!= None and a.get("product")!=None:
                seed_product_dict[a.get("locus_tag")[0]] = a.get("product")[0]
            
for feature in tb_genome_record.features:
    if not(feature.type == 'gene'):
        a = feature.qualifiers
        if a.get("locus_tag")!= None:
            seed_position_dict[a.get("locus_tag")[0]] = (int(feature.location.start),int(feature.location.end),int(feature.location.strand))

seq_ids = util.list_dirs(seed_dir)
out_list = []
for id in seq_ids:
    if os.path.exists(seed_dir + '/' + str(id) + '/rscape_1.cov'):
        with open(seed_dir + '/' + str(id) + '/rscape_1.cov', 'r') as f:  
            num_pairs = 0
            e_values = []
            for l in f:
                if (not ('#' in l)):
                    a = l.split()
                    if len(a) > 6 and abs(int(a[2])-int(a[1])) > 3:    # Exclude covarying pairs less than 3 bp apart)
                        e_values.append(float(a[4]))
                        num_pairs +=1
        if len(e_values) > 0:
            tot = sum([math.log(x) for x  in e_values])
            num = len(e_values)
            combined_e_value = 1-chi2.cdf(-2*tot, 2*num)
        else:
            combined_e_value = 999
        if combined_e_value < 1e-5:
            with open(seed_dir + '/' + str(id) + '/align_bh_3.sto', 'r') as g:  
                species_list = []
                tb_loc = 'None'
                seed_loc = 'None'
                for l in g:
        
                    if len(l) > 1 and (not ('#' in l)):
                        species_list.append(l.split()[0])
                        if tb_species in l:
                            tb_loc = (l.split()[0].split('/')[1].split('-'), l.split()[0].split('/')[2].split('-'))
                           
                        if seed_species in l:
                            seed_loc = (l.split()[0].split('/')[1].split('-'), l.split()[0].split('/')[2].split('-'))
                           
                num_species = len(set(species_list))
                if os.path.exists(seed_dir + '/' + str(id) + '/rscape_3.cacofold.R2R.sto.svg'):
                    structure_plotted = True
                    if tb_loc == 'None':
                        tb_region = 'None'
                    else:
                        tb_region = find_tb_region_info(int(tb_loc[0][0]), int(tb_loc[0][1]), int(tb_loc[1][0]), int(tb_loc[1][1]), search_region)
                    if seed_loc == 'None':
                        seed_region = 'None'
                    else:
                        seed_region = seed_position_dict[id]                                  
                                                         
                    print(id, seed_product_dict[id], num_species, num_pairs, combined_e_value, seed_loc, tb_loc, structure_plotted, seed_region, tb_region)
                    out_list.append([id, seed_product_dict[id], num_species, num_pairs, combined_e_value, structure_plotted, tb_region[0], tb_region[1]])
out_df = pd.DataFrame(out_list, columns = ['Locus', 'Product' , 'Num_Species', 'Num_Pairs', 'E_Val', 'Plotted' , 'Start', 'End'])
out_df.to_csv(project_dir + '/hit_summary.csv')

Rv0053 30S ribosomal protein S6 640 5 0.0 (['58191', '58482'], ['287', '498']) (['58191', '58482'], ['287', '498']) True (58191, 58482, 1) (57978, 58189)
Rv0414c thiamine-phosphate synthase 182 3 6.151523734843067e-12 (['500349', '501018'], ['378', '496']) (['500349', '501018'], ['378', '496']) True (500349, 501018, -1) (501022, 501140)
Rv0415 thiamine biosynthesis oxidoreductase ThiO 139 2 4.503625399054911e-06 (['501147', '502170'], ['375', '494']) (['501147', '502170'], ['375', '494']) True (501147, 502170, 1) (501022, 501141)
Rv0423c phosphomethylpyrimidine synthase 368 5 0.0 (['508580', '510225'], ['375', '497']) (['508580', '510225'], ['375', '497']) True (508581, 510225, -1) (510228, 510350)
Rv0440 molecular chaperone GroEL 631 6 0.0 (['528607', '530230'], ['219', '500']) (['528607', '530230'], ['219', '500']) True (528607, 530230, 1) (528326, 528607)
Rv0500A DNA-binding protein 250 4 8.881784197001252e-16 (['591110', '591346'], ['380', '485']) (['591110', '591346'], ['380', '48

In [29]:
id = 'Rv3848' 
if os.path.exists(seed_dir + '/' + str(id) + '/final_summary.txt'):
    with open(seed_dir + '/' + str(id) + '/final_summary.txt', 'r') as f:  
        phylum_dict = {}
        for l in f:
            if (not ('#' in l)):
                a = l.split()
                if float(a[15]) < 1e-5:
                    species = names_dict[a[0].split('/')[0]]
                    phylum = species.split(' ')[0]
                    if phylum in phylum_dict:
                        phylum_dict[phylum] = phylum_dict[phylum] + 1
                    else:
                        phylum_dict[phylum] = 1
print(phylum_dict)
                

{'Nocardia': 14, 'Rhodococcus': 11, 'Mycolicibacterium': 34, 'Mycobacterium': 19, 'Micromonospora': 6, 'Salinispora': 1, 'Actinoplanes': 4, 'Tsukamurella': 1, 'Nonomuraea': 3, 'Janibacter': 2, 'Aeromicrobium': 3, 'Phytohabitans': 1, 'Nocardioides': 8, 'Allosaccharopolyspora': 1, 'Luteipulveratus': 1, 'Actinocatenispora': 2, 'Saccharopolyspora': 2, 'Actinosynnema': 2, 'Nocardiopsis': 1, 'Raineyella': 1, 'Natronoglycomyces': 1, 'Dactylosporangium': 1, 'Streptosporangium': 1, 'Pseudonocardia': 2, 'Thermobispora': 1, 'Mycolicibacillus': 1}


In [12]:
filename_dict[ 'NZ_CP102342.1']

'GCF_024600175.1_ASM2460017v1_genomic.gbff'

In [46]:
tb_species

'NC_000962.3'